In [1]:
import csv
import logging
import os
import random
import sys

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
#from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear

from bertviz.bertviz import attention, visualization
from bertviz.bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
logger = logging.getLogger(__name__)

#bert_classifier_model_dir = "./bert_classifier/" ## Path of BERT classifier model path
#bert_classifier_model_dir = "./data/yelp/bert_classifier_2epochs/"

#bert_classifier_model_dir = "./data/lipton/sentiment/orig/bert_classifier_10epochs8b_490seqlen/"       #Apr 24
#eval_accuracy = 0.9102040816326531  and  eval_loss = 0.35673839559838655  

#bert_classifier_model_dir = "./data/lipton/sentiment/orig/bert_classifier_100epochs16b/"    #apr27
#eval_accuracy = 0.9306122448979591   and eval_loss = 0.6695167317746147

bert_classifier_model_dir = "./data/lipton/sentiment/orig/bert_classifier_100epochs8b_490seqlen/"  #Apr 24
#eval_accuracy = 0.8979591836734694   and eval_loss = 0.9967757850885384           # Try with this one <--

# QUESTION: Whats the difference between loss and acc?  in run_classifier.py

#def accuracy(out, labels):
#    outputs = np.argmax(out, axis=1)
#    return np.sum(outputs == labels)

# eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
# eval_sampler = SequentialSampler(eval_data)Run prediction for full data
# eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)
# model.eval()

# eval_loss, eval_accuracy, nb_eval_steps, nb_eval_examples = 0, 0, 0, 0

# for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader, desc="Evaluating"):
#    ...
#    tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
#    ...
#    tmp_eval_accuracy = accuracy(logits, label_ids)   #
#    ...
#    eval_loss += tmp_eval_loss.mean().item()
#    eval_accuracy += tmp_eval_accuracy
#    ...
#    nb_eval_examples += input_ids.size(0)
#    nb_eval_steps += 1
# ..
# eval_loss = eval_loss / nb_eval_steps                 <--- loss is sum of batch mean losses / nb_steps
# eval_accuracy = eval_accuracy / nb_eval_examples      <--- acc  is sum of each accuracy / nmb_examples  ( a little more fine grained)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
logger.info("device: {}, n_gpu {}".format(device, n_gpu))

In [34]:
#before doing the following I need to create sentiment_train_0 , sentiment_train_1 , etc versions of our training data
def create_classification_file(input_df, output_file_path):
    with open(output_file_path, "w") as out_fp:
        writer = csv.writer(out_fp, delimiter="\t")
        for i in tqdm(range(input_df.shape[0])):
            line = input_df.Text.values[i]
            writer.writerow([line.strip()])
            
import pandas as pd
origd = "data/lipton/sentiment/orig/bert_classifier_training/"
train_df = pd.read_table(origd+"train.csv",sep="\t",header=None,names=["Text","sentiment"])
dev_df = pd.read_table(origd+"dev.csv",sep="\t",header=None,names=["Text","sentiment"])
test_df = pd.read_table(origd+"test.csv",sep="\t",header=None,names=["Text","sentiment"])

train_neg = train_df[train_df.sentiment == 0][["Text"]]
train_pos = train_df[train_df.sentiment == 1][["Text"]]
dev_neg = dev_df[dev_df.sentiment == 0][["Text"]]
dev_pos = dev_df[dev_df.sentiment == 1][["Text"]]
test_neg = test_df[test_df.sentiment == 0][["Text"]]
test_pos = test_df[test_df.sentiment == 1][["Text"]]

create_classification_file(train_neg, origd+"sentiment_train_0.txt" )
create_classification_file(train_pos, origd+"sentiment_train_1.txt" )
create_classification_file(dev_neg, origd+"sentiment_dev_0.txt" )
create_classification_file(dev_pos, origd+"sentiment_dev_1.txt" )
create_classification_file(test_neg, origd+"sentiment_test_0.txt" )
create_classification_file(test_pos, origd+"sentiment_test_1.txt" )

100%|██████████| 245/245 [00:00<00:00, 33994.00it/s]


In [58]:
#also need to handle reference data!! 
refd = "./data/lipton/sentiment/new/"
train_df = pd.read_table(refd+"train.tsv",sep="\t")
dev_df = pd.read_table(refd+"dev.tsv",sep="\t")
test_df = pd.read_table(refd+"test.tsv",sep="\t")

rtrain_neg = train_df[train_df.Sentiment == "Negative"][["Text"]]
rtrain_pos = train_df[train_df.Sentiment == "Positive"][["Text"]]
rdev_neg = dev_df[dev_df.Sentiment == "Negative"][["Text"]]
rdev_pos = dev_df[dev_df.Sentiment == "Positive"][["Text"]]
rtest_neg = test_df[test_df.Sentiment == "Negative"][["Text"]]
rtest_pos = test_df[test_df.Sentiment == "Positive"][["Text"]]

create_classification_file(rtrain_neg, refd+"ref_sentiment_train_0.txt" )
create_classification_file(rtrain_pos, refd+"ref_sentiment_train_1.txt" )
create_classification_file(rdev_neg, refd+"ref_sentiment_dev_0.txt" )
create_classification_file(rdev_pos, refd+"ref_sentiment_dev_1.txt" )
create_classification_file(rtest_neg, refd+"ref_sentiment_test_0.txt" )
create_classification_file(rtest_pos, refd+"ref_sentiment_test_1.txt" )

100%|██████████| 243/243 [00:00<00:00, 32431.22it/s]


In [ ]:
# cat ref_sentiment_train_0.txt ref_sentiment_dev_0.txt ref_sentiment_test_0.txt > reference_0.txt
# cat ref_sentiment_train_1.txt ref_sentiment_dev_1.txt ref_sentiment_test_1.txt > reference_1.txt

# however looking at YELP.. the reference is just against the test set .. so we might just want to do it against
# ref_sentiment_test_0.txt  AND ref_sentiment_test_1.txt

In [ ]:
# CONTENTS OF BERT_CLASSIFIER_TRAINING
#diego@microdeep:~/spr20_cf_gen/TDRG/data/lipton/sentiment/orig/bert_classifier_training$ ls
#dev.csv                                   reference_1.txt      sentiment_test_0.txt   sentiment_train_1.txt  tfidf_train1.ann
#processed_files_with_bert_with_best_head  sentiment_dev_0.txt  sentiment_test_1.txt   test.csv               train.csv
#reference_0.txt                           sentiment_dev_1.txt  sentiment_train_0.txt  tfidf_train0.ann

# IMPORTANT:
# FOLDER: bert_classifier_training//processed_files_with_bert_with_best_head/   
#  --> BASED ON (9,5):  bert_classifier_10epochs8b_490seqlen/

# NOW select other BERT MODEL TO use and store info/results in its folder (ie, )


In [3]:
# file paths
#data_dir = "/home/ubuntu/bhargav/data/"
#data_dir = "/home/diego/spr20_cf_gen/TDRG/data/"
#dataset = "yelp" # amazon / yelp / imagecaption

data_dir = "data/lipton/sentiment/orig/"
dataset = "bert_classifier_training/"
refd = "./data/lipton/sentiment/new/"

train_0 = os.path.join(data_dir ,"{}/sentiment_train_0.txt".format(dataset))
train_1 = os.path.join(data_dir,"./{}/sentiment_train_1.txt".format(dataset))
test_0 = os.path.join(data_dir,"./{}/sentiment_test_0.txt".format(dataset))
test_1 = os.path.join(data_dir,"./{}/sentiment_test_1.txt".format(dataset))
dev_0 = os.path.join(data_dir,"./{}/sentiment_dev_0.txt".format(dataset))
dev_1 = os.path.join(data_dir,"./{}/sentiment_dev_1.txt".format(dataset))
reference_0 = os.path.join(refd,"ref_sentiment_test_0.txt")
reference_1 = os.path.join(refd,"ref_sentiment_test_1.txt")

In [4]:
# file paths
#data_dir = "/home/ubuntu/bhargav/data/"
#data_dir = "/home/diego/spr20_cf_gen/TDRG/data/"
#dataset = "yelp" # amazon / yelp / imagecaption

train_0_out = os.path.join(data_dir ,"./{}/processed_files_with_bert_with_best_head/sentiment_train_0.txt".format(dataset))
train_1_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/sentiment_train_1.txt".format(dataset))
test_0_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/sentiment_test_0.txt".format(dataset))
test_1_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/sentiment_test_1.txt".format(dataset))
dev_0_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/sentiment_dev_0.txt".format(dataset))
dev_1_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/sentiment_dev_1.txt".format(dataset))
reference_0_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/reference_0.txt".format(dataset))
reference_1_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/reference_1.txt".format(dataset))

In [5]:
"""
## Model for performing Classification
model_cls = BertForSequenceClassification.from_pretrained(bert_classifier_model_dir, num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model_cls.to(device)
model_cls.eval()
"""

## Model to get the attention weights of all the heads
#model = BertModel.from_pretrained(bert_classifier_model_dir)         #from bertviz.bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer

#instead: maybe I should use not the bertviz one?  
model = BertForSequenceClassification.from_pretrained(bert_classifier_model_dir, num_labels=2)
# NO  this way runs out of memory when doing attribution

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
    

In [6]:
#max_seq_len=70 # Maximum sequence length 
max_seq_len = 490
sm = torch.nn.Softmax(dim=-1) ## Softmax over the batch

In [7]:
common_words=['is','are','was','were','has','have','had','a','an','the','this','that','these','those','there','how','i','we',
             'he','she','it','they','them','their','his','him','her','us','our', 'and','in','my','your','you', 'will', 'shall']
common_words_tokens = tokenizer.convert_tokens_to_ids(common_words)
not_to_remove_ids = tokenizer.convert_tokens_to_ids(["[CLS]","[SEP]", ".", "?", "!"])
not_to_remove_ids += common_words_tokens

In [8]:
def read_file(file_path):
    with open(file_path) as fp:
        data = fp.read().splitlines()
    return data

In [42]:
def create_output_file(original_sentences,processed_sentences, output_file, sentiment="<POS>"):
    with open(output_file,"w") as fp:
        for sen1,sen2 in zip(original_sentences,processed_sentences):
            if sen1 != None and sen2 != None:
                str1 = sentiment + " <CON_START> " + sen2 + " <START> " + sen1 + " <END>\n"
                fp.write(str1)

In [43]:
def create_ref_output_file(processed_sentences, output_file, sentiment="<POS>"):
    with open(output_file,"w") as fp:
        for sen in tqdm(processed_sentences):
            if sen != None:
                str1 = sentiment + " <CON_START> " + sen + " <START>\n"
                fp.write(str1)

In [44]:
def concate_files(inp_files, out_files):
    with open(out_files,"w") as fp:
        for file in inp_files:
            with open(file) as f:
                for line in f:
                    fp.write(line)

In [9]:
def run_attn_examples(input_sentences, layer, head, bs=128):
    """
    Returns Attention weights for selected Layer and Head along with ids and tokens
    of the input_sentence
    """
    ids = []
    ids_to_decode = [None for k in range(len(input_sentences))]
    tokens_to_decode = [None for k in range(len(input_sentences))]
    segment_ids = []
    input_masks = []
    attention_weights = [None for z in input_sentences]
    ## BERT pre-processing
    for j,sen in enumerate(tqdm(input_sentences)):
        
        text_tokens = tokenizer.tokenize(sen)
        if len(text_tokens) >= max_seq_len-2:
            text_tokens = text_tokens[:max_seq_len-4]
        tokens = ["[CLS]"] + text_tokens + ["[SEP]"]
        tokens_to_decode[j] = tokens
        temp_ids = tokenizer.convert_tokens_to_ids(tokens)
        ids_to_decode[j] = temp_ids
        input_mask = [1] * len(temp_ids)
        segment_id = [0] * len(temp_ids)
        padding = [0] * (max_seq_len - len(temp_ids))
        
        
        temp_ids += padding
        input_mask += padding
        segment_id += padding
        
        ids.append(temp_ids)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    
    # Convert Ids to Torch Tensors
    ids = torch.tensor(ids) 
    segment_ids = torch.tensor(segment_ids)
    input_masks = torch.tensor(input_masks)
    
    steps = len(ids) // bs
    
    for i in trange(steps+1):
        if i == steps:
            temp_ids = ids[i * bs : len(ids)]
            temp_segment_ids = segment_ids[i * bs: len(ids)]
            temp_input_masks = input_masks[i * bs: len(ids)]
        else:
            temp_ids = ids[i * bs : i * bs + bs]
            temp_segment_ids = segment_ids[i * bs: i * bs + bs]
            temp_input_masks = input_masks[i * bs: i * bs + bs]
        
        temp_ids = temp_ids.to(device)
        temp_segment_ids = temp_segment_ids.to(device)
        temp_input_masks = temp_input_masks.to(device)
        with torch.no_grad():
             _, _, attn = model(temp_ids, temp_segment_ids, temp_input_masks)
        # Concate Attention weights
        for j in range(len(attn[layer]['attn_probs'])):
            attention_weights[i * bs + j] = (attn[layer]['attn_probs'][j][head][0]).to('cpu')
    
    return attention_weights, ids_to_decode, tokens_to_decode

In [10]:
def prepare_data(aw, ids_to_decode, tokens_to_decode):
    out_sen = [None for i in range(len(aw))]
    for i in trange(len(aw)):        
        topv, topi = aw[i].topk(ids_to_decode[i].index(0))
        topi = topi.tolist()
        topv = topv.tolist()

        #print("Original Top Indexes = {}".format(topi))
        topi = [topi[j] for j in range(len(topi)) if ids_to_decode[i][topi[j]] not in not_to_remove_ids] # remove noun and common words
        #print("After removing Nouns = {}".format(topi))
        
        topi = [topi[j] for j in range(len(topi)) if "##" not in tokens_to_decode[i][topi[j]]] # Remove half words
        #print("After removing Half-words = {}".format(topi))

        # DIEGO:   WHAT IS THIS DOING ? 
        if (len(topi) < 4 and len(topi) > 0):
            topi = [topi[0]]
        elif(len(topi) < 8):
            topi = topi[:2]
        else:
            topi = topi[:3]

        #print("Final Topi = {}".format(topi))
        final_indexes = []
        count = 0
        count1 = 0
        #print(ids_to_decode[i], tokens_to_decode[i])
        while ids_to_decode[i][count] != 0:
            if count in topi:
                while ids_to_decode[i][count + count1 + 1] != 0:
                    if "##" in tokens_to_decode[i][count + count1 + 1]:
                        count1 += 1
                    else:
                        break
                count += count1
                count1 = 0
            else:
                final_indexes.append(ids_to_decode[i][count])
            count += 1

        #print(final_indexes)
        temp_out_sen = tokenizer.convert_ids_to_tokens(final_indexes)
        temp_out_sen = " ".join(temp_out_sen).replace(" ##", "").replace("[CLS]","").replace("[SEP]","")
        #print(temp_out_sen, "\n\n")
        out_sen[i] = temp_out_sen.strip()
    
    return out_sen

In [11]:
train_0_data = read_file(train_0)
train_1_data = read_file(train_1)
dev_0_data = read_file(dev_0)
dev_1_data = read_file(dev_1)
test_0_data = read_file(test_0)
test_1_data = read_file(test_1)
ref_0_data = read_file(reference_0)
ref_1_data = read_file(reference_1)

In [49]:
#generate files but before hand create folder data/yelp/processed_files_with_bert_with_best_head/
!cd data/lipton/sentiment/orig/bert_classifier_training/; mkdir processed_files_with_bert_with_best_head; ls

dev.csv					  sentiment_test_0.txt	 test.csv
processed_files_with_bert_with_best_head  sentiment_test_1.txt	 train.csv
sentiment_dev_0.txt			  sentiment_train_0.txt
sentiment_dev_1.txt			  sentiment_train_1.txt


In [50]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(train_0_data, layer=9, head=5, bs=16)
train_0_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(train_0_data, train_0_out_sen, train_0_out, sentiment="<NEG>")

100%|██████████| 851/851 [00:00<00:00, 3949.22it/s]


In [51]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(train_1_data, layer=9, head=5, bs=16)
train_1_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(train_1_data, train_1_out_sen, train_1_out, sentiment="<POS>")

100%|██████████| 856/856 [00:00<00:00, 3908.56it/s]


In [52]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(dev_0_data, layer=9, head=5, bs=16)
dev_0_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(dev_0_data, dev_0_out_sen, dev_0_out, sentiment="<NEG>")

100%|██████████| 122/122 [00:00<00:00, 3514.84it/s]


In [53]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(dev_1_data, layer=9, head=5, bs=16)
dev_1_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(dev_1_data, dev_1_out_sen, dev_1_out, sentiment="<POS>")

100%|██████████| 123/123 [00:00<00:00, 3942.59it/s]


In [54]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(test_1_data, layer=9, head=5, bs=16)
test_1_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(test_1_data, test_1_out_sen, test_1_out, sentiment="<POS>")

100%|██████████| 245/245 [00:00<00:00, 3975.24it/s]


In [55]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(test_0_data, layer=9, head=5, bs=16)
test_0_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(test_0_data, test_0_out_sen, test_0_out, sentiment="<NEG>")

100%|██████████| 243/243 [00:00<00:00, 3778.36it/s]


In [ ]:
# HAD TO DO REF work

In [63]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(ref_1_data, layer=9, head=5, bs=16)
ref_1_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
#create_ref_output_file(ref_1_data, ref_1_out_sen, reference_1_out, sentiment="<NEG>")
#--> TypeError: create_ref_output_file() got multiple values for argument 'sentiment'

#doesn't need original ref_1_data 
# def create_ref_output_file(processed_sentences, output_file, sentiment="<POS>"):
create_ref_output_file(ref_1_out_sen, reference_1_out, sentiment="<NEG>")

100%|██████████| 243/243 [00:00<00:00, 277488.67it/s]


In [64]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(ref_0_data, layer=9, head=5, bs=16)
ref_0_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
#create_ref_output_file(ref_0_data, ref_0_out_sen, reference_0_out, sentiment="<POS>")
create_ref_output_file(ref_0_out_sen, reference_0_out, sentiment="<POS>")

100%|██████████| 245/245 [00:00<00:00, 278785.81it/s]


In [ ]:
# COMBINE _0  and _1 files WHEN YOU GET BACK

# diego@microdeep:~/spr20_cf_gen/TDRG/data/lipton/sentiment/orig/bert_classifier_training/processed_files_with_bert_with_best_head$ cat sentiment_train_0.txt sentiment_train_1.txt > sentiment_train.txt
# diego@microdeep:~/spr20_cf_gen/TDRG/data/lipton/sentiment/orig/bert_classifier_training/processed_files_with_bert_with_best_head$ cat sentiment_dev_0.txt sentiment_dev_1.txt > sentiment_dev.txt

### Integrated Gradients

In [11]:
# MOVE TO Head_selection bottom when done
# integradted gradients / expected gradients / integrated hessians

# IG using captum
# pip install captum
# https://captum.ai/tutorials/IMDB_TorchText_Interpret  aND  https://captum.ai/docs/extension/integrated_gradients
#!pip install captum

In [17]:
# TAKE 1 :   NOPE
#https://github.com/huggingface/transformers/blob/master/notebooks/02-transformers.ipynb

tokens = tokenizer.tokenize(train_0_data[0])
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
#tokens_ids = tokenizer.build_inputs_with_special_tokens(tokens_ids)  # Add the required special tokens
tokens_pt = torch.tensor([tokens_ids])
print("Tokens: {}".format(tokens))
print("\nTokens id: {}".format(tokens_ids))
print("\nTokens PyTorch: {}".format(tokens_pt))
#outputs, pooled = model(tokens_pt)  ## Now we're ready to go through BERT with out input  # <--- error
#print("Token wise output: {}, Pooled output: {}".format(outputs.shape, pooled.shape))

Tokens: ['long', ',', 'boring', ',', 'b', '##las', '##ph', '##em', '##ous', '.', 'never', 'have', 'i', 'been', 'so', 'glad', 'to', 'see', 'ending', 'credits', 'roll', '.']

Tokens id: [2146, 1010, 11771, 1010, 1038, 8523, 8458, 6633, 3560, 1012, 2196, 2031, 1045, 2042, 2061, 5580, 2000, 2156, 4566, 6495, 4897, 1012]

Tokens PyTorch: tensor([[ 2146,  1010, 11771,  1010,  1038,  8523,  8458,  6633,  3560,  1012,
          2196,  2031,  1045,  2042,  2061,  5580,  2000,  2156,  4566,  6495,
          4897,  1012]])


In [46]:
# TAKE 2 ( based on above method for preping example.. not sure output is right though)
def prep_example(input_sentence, in_label=0):
    """
    Returns inputs nessary for model to process a single input_sentence and its label ( 0 / 1 )
    """
    debug = False
    bs = 1
    ids = []
    ids_to_decode = [None]
    tokens_to_decode = [None]
    segment_ids = []
    input_masks = []
    attention_weights = [None]
    sen = input_sentence
    j = 0
    max_seq_len = 490
    text_tokens = tokenizer.tokenize(sen)
    if len(text_tokens) >= max_seq_len-2:
        text_tokens = text_tokens[:max_seq_len-4]
        
    tokens = ["[CLS]"] + text_tokens + ["[SEP]"]
    tokens_to_decode[j] = tokens
    temp_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    ids_to_decode[j] = temp_ids
    input_mask = [1] * len(temp_ids)
    segment_id = [0] * len(temp_ids)
    padding = [0] * (max_seq_len - len(temp_ids))

    temp_ids += padding
    input_mask += padding
    segment_id += padding

    ids.append(temp_ids)
    input_masks.append(input_mask)
    segment_ids.append(segment_id)
    
 
    # Convert Ids to Torch Tensors
    ids = torch.tensor(ids) 
    segment_ids = torch.tensor(segment_ids)
    input_masks = torch.tensor(input_masks)
    input_label = torch.tensor([in_label])
    
    steps = len(ids) // bs
    if debug:
        print(ids)
        print(input_masks)
        print(segment_ids)
        print(input_label)
        print(steps)
        
    #for i in trange(steps+1):
    for i in range(1):
        if i == steps:
            temp_ids = ids[i * bs : len(ids)]
            temp_segment_ids = segment_ids[i * bs: len(ids)]
            temp_input_masks = input_masks[i * bs: len(ids)]
        else:
            temp_ids = ids[i * bs : i * bs + bs]
            temp_segment_ids = segment_ids[i * bs: i * bs + bs]
            temp_input_masks = input_masks[i * bs: i * bs + bs]
        
        temp_ids = temp_ids.to(device)
        temp_segment_ids = temp_segment_ids.to(device)
        temp_input_masks = temp_input_masks.to(device)
        input_label = input_label.to(device)

    return temp_ids, temp_segment_ids, temp_input_masks, input_label

In [9]:
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

"""
input_sentence = train_0_data[0]  #negative
temp_ids, temp_segment_ids, temp_input_masks, temp_input_label_ids = prep_example(input_sentence, 0)
with torch.no_grad():
    #a, b, attn = model(temp_ids, temp_segment_ids, temp_input_masks)
    
    eval_loss = model(temp_ids, temp_segment_ids, temp_input_masks, temp_input_label_ids)
    print("Eval Loss",type(eval_loss), len(eval_loss))
    logits = model(temp_ids, temp_segment_ids, temp_input_masks)
    print("Logits", type(logits), len(logits),len(logits[0]), len(logits[1]), len(logits[2]))
    
    # there is some error ( i should probably have followed eval_bert more so?)   
    
    #HERE HERE HERE  #MAKE SURE YOU ARE USING THE RIGHT MODEL (( should it be model or model_cls ?))
    
    
    #logits = logits.detach().cpu().numpy() 
    #label_ids = temp_input_label_ids.to('cpu').numpy()
    #eval_accuracy = accuracy(logits, label_ids)
    
    #loss = eval_loss[0].mean().item()
    #print(loss)   #-0.017504552379250526

    #loss = eval_loss[1].mean().item()
    #print(loss)   # -0.017707472667098045
"""
print("")

In [10]:
#print("Temp IDs:", temp_ids)  #[[  101,  2146,  1010, 11771,  1010,  1038,  8523,  8458,  6633,  3560, 1012,  2196,  2031,  1045,  2042,  2061,  5580,  2000,  2156,  4566, 6495,  4897,  1012,   102,     0,
#print("Temp segments:",temp_segment_ids)   #[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
#print("Temp input mask::",temp_input_masks) #[[1, 1, 1, 1, 1, 1, 
print("")

In [30]:
#model = BertModel.from_pretrained(bert_classifier_model_dir)
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
#model.to(device)
#model.eval()

membeds = model.embeddings

pytorch_pretrained_bert.modeling.BertEmbeddings

In [12]:
#so both model loadings are the same, and both are slightly different than Captum Tutorial

#model = BertModel.from_pretrained(bert_classifier_model_dir) 
print(type(model.embeddings))  #here its        <class 'bertviz.bertviz.pytorch_pretrained_bert.modeling.BertEmbeddings'>
print([k for k in dir(model.embeddings) if not k.startswith('_')])
"""
['LayerNorm', 
 'add_module', 'apply', 'buffers', 'children', 'cpu', 'cuda', 'double', 'dropout', 'dump_patches', 'eval', 
 'extra_repr', 'float', 'forward', 'half', 'load_state_dict', 'modules', 'named_buffers', 
 'named_children', 'named_modules', 'named_parameters', 'parameters', 'position_embeddings', 
 'register_backward_hook', 'register_buffer', 'register_forward_hook', 'register_forward_pre_hook', 'register_parameter', 
 'requires_grad_', 'share_memory', 'state_dict', 'to', 'token_type_embeddings', 
 'train', 'training', 'type', 'word_embeddings', 'zero_grad']
"""


#model = BertForSequenceClassification.from_pretrained(bert_classifier_model_dir, num_labels=2)
# this way model doesn't have model.embeddings, but it does have model.bert.embeddings
#print(type(model.bert.embeddings))         #pytorch_pretrained_bert.modeling.BertEmbeddings
#print([k for k in dir(model.bert.embeddings) if not k.startswith('_')])
"""
['LayerNorm', 
 'add_module', 'apply', 'buffers', 'children', 'cpu', 'cuda', 'double', 'dropout', 'dump_patches', 'eval', 
 'extra_repr', 'float', 'forward', 'half', 'load_state_dict', 'modules', 'named_buffers', 
 'named_children', 'named_modules', 'named_parameters', 'parameters', 'position_embeddings', 
 'register_backward_hook', 'register_buffer', 'register_forward_hook', 'register_forward_pre_hook', 'register_parameter', 
 'requires_grad_', 'share_memory', 'state_dict', 'to', 'token_type_embeddings', 
 'train', 'training', 'type', 'word_embeddings', 'zero_grad']
"""


#whereas in tutorial model.embeddings is 
#print("Model Embedding", type(model.embedding))  #torch.nn.modules.sparse.Embedding
#print([k for k in dir(model.embedding) if not k.startswith('_')])
"""
['add_module', 'apply', 'buffers', 'children', 'cpu', 'cuda', 'double', 'dump_patches', 'embedding_dim', 'eval', 
 'extra_repr', 'float', 'forward', 'from_pretrained', 'half', 'load_state_dict', 'max_norm', 'modules', 'named_buffers', 
 'named_children', 'named_modules', 'named_parameters', 'norm_type', 'num_embeddings', 'padding_idx', 'parameters', 
 'register_backward_hook', 'register_buffer', 'register_forward_hook', 'register_forward_pre_hook', 'register_parameter', 
 'requires_grad_', 'reset_parameters', 'scale_grad_by_freq', 'share_memory', 'sparse', 'state_dict', 'to', 'train', 
 'training', 'type', 'weight', 'zero_grad']
"""
print("")

<class 'bertviz.bertviz.pytorch_pretrained_bert.modeling.BertEmbeddings'>
['LayerNorm', 'add_module', 'apply', 'buffers', 'children', 'cpu', 'cuda', 'double', 'dropout', 'dump_patches', 'eval', 'extra_repr', 'float', 'forward', 'half', 'load_state_dict', 'modules', 'named_buffers', 'named_children', 'named_modules', 'named_parameters', 'parameters', 'position_embeddings', 'register_backward_hook', 'register_buffer', 'register_forward_hook', 'register_forward_pre_hook', 'register_parameter', 'requires_grad_', 'share_memory', 'state_dict', 'to', 'token_type_embeddings', 'train', 'training', 'type', 'word_embeddings', 'zero_grad']



In [16]:

#from tutorial
#PAD_IND = TEXT.vocab.stoi['pad']
#token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
#reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)
#print("Refer Indices",type(reference_indices), reference_indices)
#<class 'torch.Tensor'> tensor([[6978, 6978, 6978, 6978, 6978, 6978, 6978]])

#so i just need to make a tensor of the right size with PAD as the reference indice
#torch.tensor(indexed, device=device)
#pad_ids = tokenizer.convert_tokens_to_ids(["[PAD]"])  
pad_ids = tokenizer.convert_tokens_to_ids(["[SEP]"])    #[102]
print(pad_ids)  #[0]  .. ["[PAD]"]

[102]


In [14]:
# TAKE 3 based on eval_bert.py
from eval_bert import *
# print(logging.getLoggerClass().root.handlers[0].baseFilename)
# /home/diego/spr20_cf_gen/TDRG/logs/eval_bert.txt

#from captum.attr import IntegratedGradients
from captum.attr import LayerIntegratedGradients, visualization

#lig = LayerIntegratedGradients(model, model.embeddings)       #BertModel.from_pretrained
lig = LayerIntegratedGradients(model, model.bert.embeddings)    #BertForSequenceClassification ( this way runs out of memory !!)

def pred_sentence(input_sentence, input_labels=["0"]):
    
    processors = { "yelp": YelpProcessor, }
    num_labels_task = { "yelp": 2, }
    task_name = "yelp"
    processor = processors[task_name]()
    num_labels = num_labels_task[task_name]
    label_list = processor.get_labels()

    """ # Don't Reload Model cause it takes up too much GPU !!
    
    bc_model_dir = "./data/lipton/sentiment/orig/bert_classifier_100epochs8b_490seqlen/"  #Apr 24
    output_config_file = os.path.join(bc_model_dir, "bert_config.json")
    output_model_file = os.path.join(bc_model_dir, "pytorch_model.bin")

    config = BertConfig(output_config_file)
    model = BertForSequenceClassification(config, num_labels=num_labels)  #
    model.load_state_dict(torch.load(output_model_file))

    model.to(device)
    """

    # change Diego
    #eval_examples = processor.get_dev_examples(args.data_dir)
    lines = []
    for i in range(len(input_sentence)):
        lines.append([input_sentence[i], input_labels[i]])

    eval_examples = processor._create_examples(lines,set_type="dev")
    max_seq_len = 490
    eval_features = convert_examples_to_features(eval_examples, label_list, max_seq_len, tokenizer)
    
    eval_batch_size = 16   #maybe make this 1? 
    
    all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    # Run prediction for full data
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)
    
    #reference_ids = torch.tensor(np.zeros(input_ids.shape, dtype=int))
    print(len(lines))
    reference_ids = torch.tensor(np.zeros((len(lines),490), dtype=int))
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    ret_preds = []
    ret_attributions = [] 
    
    # attribution score will be computed with respect to target class
    # applying integrated gradients on the SoftmaxModel and input data point
    
    # ig = IntegratedGradients(model)
    # 
    # https://captum.ai/tutorials/IMDB_TorchText_Interpret  
    # <-- Do this after you finish ICML stuff   HERE
    
    
    # then do similar thing you did but with attributes being removed at different thresholds ( make this a list ) 
    # compare this with attention results ( for both Yelp and Lipton for various thresholds )
    # use auto metrics
    #
    # then move back to B-GST
    #   make similar list of results (train/dev/test) for attention vs ig methods vs expected gradients

    # ig = IntegratedGradients(model)

    print("Load LayerIntegratedGradient")
    #lig = LayerIntegratedGradients(model, model.embeddings)       #BertModel.from_pretrained
    #lig = LayerIntegratedGradients(model, model.bert.embeddings)    #BertForSequenceClassification ( this way runs out of memory !!)

    # from constantly reloading model
    #RuntimeError: CUDA out of memory. Tried to allocate 5.37 GiB (GPU 0; 11.91 GiB total capacity; 10.41 GiB already allocated; 838.25 MiB free; 10.48 GiB reserved in total by PyTorch)
    attrbs = []
    
    for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader, desc="Evaluating"):
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)
        reference_ids = reference_ids.to(device)

        print("Eval:")#",input_ids, segment_ids, input_mask, label_ids)
        with torch.no_grad():
            tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
            print("Eval Loss")
            logits = model(input_ids, segment_ids, input_mask)
            print("Eval Logits")
            
        #print(type(logits),  type(logits[0]),   type(logits[1]),  type(logits[2]),len(logits)) 
        #    <class 'tuple'> <class 'list'> <class 'torch.Tensor'> <class 'list'>  3
        #print(len(logits[0]),logits[1].shape, len(logits[2]))  #12 torch.Size([1, 768]) 12
        #print(type(logits[0][0]))  #<class 'torch.Tensor'>
        #print(dir(logits[1]))
        #print(logits[1].cpu().numpy())
        #print(np.argmax(logits[1], axis=1))
        #print(logits[2])
        
        logits = logits.detach().cpu().numpy()     #BertSequenceForClassification:  
        #logits = logits[1].cpu().numpy()          #FIX if using BertModel.from_pretrained(bert_classifier_model_dir)
        
        label_ids_np = label_ids.to('cpu').numpy()

        print(logits.shape, logits)  #BertForSequenceClassification --> (1, 2)
        #print(logits.shape) #BertModel.from_pretrained --> (1, 768)
        print(label_ids, label_ids_np)    #[0]
        tmp_eval_accuracy = accuracy(logits, label_ids_np)
        ret_preds.append(np.argmax(logits, axis=1).flatten())
        
        print(tmp_eval_accuracy,label_ids,ret_preds)  # 0 [0] [array([77])]
        
        print(type(tmp_eval_loss))
        eval_loss += tmp_eval_loss.mean().item()  #'tuple' object has no attribute 'mean'
        eval_accuracy += tmp_eval_accuracy


        # maybe take Confidence scores as well ? 
        
        #print(input_ids)  #tensor([[  101,  2146,  1010, 11771,  1038,  ..., 0, ], [  101,  2025,  2204, 999, ...,0] ])
        #print(lines)      #[['Long, boring, blasphemous. ..', '0'], ['Not good! Rent ..', '0']]
        #print(label_ids)  #[0 0]
        #attributions, approx_error = ig.attribute(input, target=target_class_index, return_convergence_delta=True)
        
        
        #attributions, approx_error = ig.attribute(input_ids, target=0, return_convergence_delta=True)
        # RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; 
        #  but got torch.cuda.FloatTensor instead (while checking arguments for embedding)
        
        #attributions, approx_error = ig.attribute(input_ids, target=label_ids, return_convergence_delta=True)
        #RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; 
        #but got torch.cuda.FloatTensor instead (while checking arguments for embedding)
        #attributions, approx_error = ig.attribute((input_ids, segment_ids, input_mask), target=label_ids, return_convergence_delta=True)
        
        # https://captum.ai/tutorials/IMDB_TorchText_Interpret  
        # if still not working, after class actually do the tutorial and look at what type/sizes inputs/refs are  ! HERE
        #attributions_ig, delta = lig.attribute(input_ids, reference_indices, n_steps=500, return_convergence_delta=True)
        
        print("Now do attribution!")
        #attributions_ig, delta = lig.attribute(input_ids, n_steps=500, return_convergence_delta=True)  #out of memory
        #attributions_ig, delta = lig.attribute(input_ids, reference_ids, n_steps=10, return_convergence_delta=True)
        #AssertionError: Target not provided when necessary, cannot take gradient with respect to multiple outputs.
        
        print(input_ids.shape)  #torch.Size([1, 490])
        #reference_ids = torch.tensor(np.zeros(max_seq_len))
        #reference_ids = torch.tensor(np.zeros(input_ids.shape, dtype=int))
        #reference_ids = reference_ids.to(device)
        print(reference_ids.shape)#torch.Size([1, 490])
        
        #print(input_ids)  #tensor([[  101,  2146,  1010, 11771,  1010,  1038,  8523,
        #print(reference_ids)  #tensor([[0., 0., 0., 0., 0.
        
        attributions_ig, delta = lig.attribute(inputs = input_ids, 
                                               baselines = reference_ids, 
                                               target = label_ids,
                                               n_steps=10, 
                                               return_convergence_delta=True)
        #RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.cuda.DoubleTensor instead (while checking arguments for embedding)
        #fixed scalar, now getting out of memory :(
        #RuntimeError: CUDA out of memory. Tried to allocate 58.00 MiB (GPU 0; 11.91 GiB total capacity; 10.82 GiB already allocated; 10.25 MiB free; 11.29 GiB reserved in total by PyTorch)
        # OR
        #AssertionError: Target not provided when necessary, cannot take gradient with respect to multiple outputs.

        # https://github.com/pytorch/captum/blob/5f0485e8bd63cbcdb9b7d814905618235117f7c0/captum/attr/_core/layer/layer_integrated_gradients.py#L25
        
        #from Tutorial
        #attributions_ig, delta = lig.attribute(input_indices, reference_indices, n_steps=500, return_convergence_delta=True)

        
        
        #print("Lines",len(lines))
        #print("InputIDs",len(input_ids))
        nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_examples
    #loss = tr_loss/nb_tr_steps if args.do_train else None
    result = {'eval_loss': round(eval_loss,5), 'eval_accuracy': round(eval_accuracy,5), 'steps': nb_eval_steps, 
              'examples': nb_eval_examples, 'preds': ret_preds, 'attributions_ig': attributions_ig}
    return result
    

input_sentence = [train_0_data[i] for i in range(1)]
input_label = ["0","0"]
print("Input: ",input_label,input_sentence)
sen_pred = pred_sentence(input_sentence, input_label)
print("Returned: ",sen_pred)
#print("Predicted", np.argmax(sen_pred["logits"], axis=1))
print("Predicted", sen_pred['preds'])

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Input:  ['0', '0'] ['Long, boring, blasphemous. Never have I been so glad to see ending credits roll.']
1
Load LayerIntegratedGradient
Eval:
Eval Loss
Eval Logits
(1, 2) [[ 7.268938 -7.370504]]
tensor([0], device='cuda:0') [0]
1 tensor([0], device='cuda:0') [array([0])]
<class 'torch.Tensor'>
Now do attribution!
torch.Size([1, 490])
torch.Size([1, 490])


Evaluating: 100%|██████████| 1/1 [00:00<00:00,  1.35it/s]

Returned:  {'eval_loss': 0.0, 'eval_accuracy': 1.0, 'steps': 1, 'examples': 1, 'preds': [array([0])], 'attributions_ig': tensor([[[-2.1389e-03, -7.1430e-03, -4.7225e-05,  ...,  8.9575e-04,
           3.4303e-03,  4.5720e-04],
         [ 9.4241e-05, -4.4118e-05,  6.4583e-04,  ..., -4.7069e-04,
          -2.8796e-04, -1.9963e-04],
         [ 1.0441e-04,  7.6457e-04,  4.4016e-04,  ..., -8.5060e-04,
           2.0069e-04, -1.1677e-04],
         ...,
         [ 0.0000e+00, -0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          -0.0000e+00, -0.0000e+00],
         [ 0.0000e+00, -0.0000e+00, -0.0000e+00,  ...,  0.0000e+00,
          -0.0000e+00, -0.0000e+00]]], device='cuda:0', grad_fn=<MulBackward0>)}
Predicted [array([0])]


In [13]:
# NOW SHOW ATTRIBUTION PER IG
# AND COMPARE WITH OTHER BRUTE FORCE TECHNIQUE

#from tutorial
vis_data_records_ig = []

def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    # attributions_ig, delta = lig.attribute(input_indices, reference_indices,  n_steps=500, return_convergence_delta=True)
    # text is list of text tokens, 
    # pred is between 0 and 1 prob score, 
    # pred_ind is 0/1, 
    # label is pos/neg
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            Label.vocab.itos[pred_ind],
                            Label.vocab.itos[label],
                            Label.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))


add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)

print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)


NameError: name 'attributions_ig' is not defined

In [ ]:
### EVALUATE TRAIN/DEV/TEST data accuracies for trained lipton data based BERT MODEL

In [48]:
# NOW CHECK EVERYTHING HERE ( where do we err and where do we do well )

lists = {"train0" : train_0_data, "train1": train_1_data, "dev0": dev_0_data, "dev1": dev_1_data, "test0": test_0_data, "test1": test_1_data, "ref0": ref_0_data, "ref1": ref_1_data}
lists_preds = {}
for k in list(lists.keys()):
    print("Evaluating ",k)
    cur = lists[k]
    lab = "0" if "0" in k else "1"
    num_ex = len(cur)
    sub_train = [cur[i] for i in range(num_ex)]
    sub_labels = [lab for i in range(num_ex)]
    temp_preds = pred_sentence(sub_train, sub_labels)
    lists_preds[k] = temp_preds
    print(k," Returned: ",temp_preds['eval_accuracy'],temp_preds['examples'])      #{'eval_loss': 0.0, 'eval_accuracy': 1.0, 'steps': 54, 'examples': 856 .. 

Evaluating  train0


Evaluating: 100%|██████████| 54/54 [00:32<00:00,  1.65it/s]


train0  Returned:  1.0 851
Evaluating  train1


Evaluating: 100%|██████████| 54/54 [00:35<00:00,  1.53it/s]


train1  Returned:  1.0 856
Evaluating  dev0


...

test1  Returned:  0.8898 245
Evaluating  ref0


Evaluating: 100%|██████████| 16/16 [00:10<00:00,  1.57it/s]


ref0  Returned:  0.88163 245
Evaluating  ref1


Evaluating: 100%|██████████| 16/16 [00:10<00:00,  1.59it/s]

ref1  Returned:  0.86831 243


In [ ]:
#NOW DO INTEGRATED GRADIENTS STUFF

#https://captum.ai/docs/extension/integrated_gradients
from captum.attr import IntegratedGradients

# attribution score will be computed with respect to target class
target_class_index = 5

# applying integrated gradients on the SoftmaxModel and input data point
ig = IntegratedGradients(model)
attributions, approx_error = ig.attribute(input, target=target_class_index, return_convergence_delta=True)